# DAY_10
***

## 02. 통계분석

#### Q1) FIFA데이터에서 각 선수의 키는 Heghit변수에 피트와 인치로 입력되어 있습니다. 
#### 이를 cm로 변환하여 새로운 변수 Height_cm을 생성하시오. 
#### ( “ ' ” 앞의 숫자는 피트이며, “ ' ” 뒤의 숫자는 인치, 1피트 = 30cm, 1인치 = 2.5cm)

In [307]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
import time

In [308]:
fifa = pd.read_csv("C:/Users/kcisa/Desktop/ADP실기/모의고사 1회/FIFA.csv", encoding = 'cp949')

In [309]:
fifa.head()

,ID,Name,Age,Nationality,Overall,Club,Preferred_Foot,Work_Rate,Position,Jersey_Number,Contract_Valid_Until,Height,Weight_lb,Release_Clause,Value,Wage
0,158023,L. Messi,31,Argentina,94,FC Barcelona,Left,Medium/ Medium,RF,10,2021,5'7,159,226500,110500,565
1,20801,Cristiano Ronaldo,33,Portugal,94,Juventus,Right,High/ Low,ST,7,2022,6'2,183,127100,77000,405
2,190871,Neymar Jr,26,Brazil,92,Paris Saint-Germain,Right,High/ Medium,LW,10,2022,5'9,150,228100,118500,290
3,193080,De Gea,27,Spain,91,Manchester United,Right,Medium/ Medium,GK,1,2020,6'4,168,138600,72000,260
4,192985,K. De Bruyne,27,Belgium,91,Manchester City,Right,High/ High,RCM,7,2023,5'11,154,196400,102000,355


In [310]:
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16642 entries, 0 to 16641
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    16642 non-null  int64 
 1   Name                  16642 non-null  object
 2   Age                   16642 non-null  int64 
 3   Nationality           16642 non-null  object
 4   Overall               16642 non-null  int64 
 5   Club                  16642 non-null  object
 6   Preferred_Foot        16642 non-null  object
 7   Work_Rate             16642 non-null  object
 8   Position              16642 non-null  object
 9   Jersey_Number         16642 non-null  int64 
 10  Contract_Valid_Until  16642 non-null  int64 
 11  Height                16642 non-null  object
 12  Weight_lb             16642 non-null  int64 
 13  Release_Clause        16642 non-null  int64 
 14  Value                 16642 non-null  int64 
 15  Wage                  16642 non-null

In [311]:
fifa['Height_cm'] = fifa.Height.str.split("'").str[0].astype('int') * 30 + fifa.Height.str.split("'").str[1].astype('int') * 2.5

'를 기준으로 split한 후 앞 문자를 숫자로 바꾸어 30을 곱하고 뒷문자를 숫자로 바꾸고 2.5를 곱한 후 두 수를 더한다.
***
#### Q2) 포지션을 의미하는 Position변수를 아래 표를 참고하여 “Forward”, “Midfielder”, 
####      “Defender”, “GoalKeeper”로 재범주화하고, factor형으로 변환하여 Position_Class 
####       라는 변수를 생성하고 저장하시오.

In [312]:
def pos_cat(x) :
    if x in ["LS", "ST", "RS", "LW", "LF", "CF", "RF", "RW"] :
        return 'Forward'
    elif x in ["LAM", "CAM", "RAM", "LM", "LCM", "CM", "RCM", "RM"] :
        return 'Midfielder'
    elif x in ["LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB", "CB", "RCB", "RB"] :
        return 'Defender'
    else :
        return 'GoalKeeper'

In [313]:
fifa['Position_Class'] = fifa.Position.apply(pos_cat)
fifa.Position_Class = fifa.Position_Class.astype('category') # factor형으로 변경

***
#### Q3) 새로 생성한 Position_Class 변수의 각 범주에 따른 Value(선수의 시장가치)의 
####      평균값의 차이를 비교하는 일원배치 분산분석을 수행하고 결과를 해석하시오. 
####      그리고 평균값의 차이가 통계적으로 유의하다면 사후검정을 수행하고 설명하시오.

In [314]:
model = ols('Value ~ C(Position_Class)', fifa).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(Position_Class),3.0,4.081181e+09,1.360394e+09,41.873906,5.988667e-27
Residual,16638.0,5.405330e+11,3.248786e+07,NaN,NaN


H0 : 모든 포지션에 대해 value의 평균은 같다.<br>
H1 : 적어도 하나의 포지션의 values의 평균은 차이가 있다.<br>
SSA의 자유도는 3(범주수 - 1)이고 SST의 자유도는 16338(관측치 개수 - 범주 수)임을 알 수 있다. 또한 p-value가 0.05 보다 작으므로 귀무가설을 기각한다. 따라서 각 포지션별 value의 평균은 차이가 있다고 볼 수 있다. 

In [315]:
posthoc = pairwise_tukeyhsd(fifa['Value'], fifa['Position_Class'], alpha=0.05)
print(posthoc)

        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
  group1     group2    meandiff  p-adj    lower      upper    reject
--------------------------------------------------------------------
  Defender    Forward    930.373  0.001   610.7429  1250.0031   True
  Defender GoalKeeper  -507.3848 0.0034  -887.6261  -127.1436   True
  Defender Midfielder   760.8787  0.001   486.7123   1035.045   True
   Forward GoalKeeper -1437.7579  0.001 -1865.9234 -1009.5923   True
   Forward Midfielder  -169.4944 0.5609  -506.9991   168.0104  False
GoalKeeper Midfielder  1268.2635  0.001   872.8782  1663.6488   True
--------------------------------------------------------------------


사후분석에서는 각 집단별간의 차이가 있는지 살펴본다. 우선 'defender'와 'forward'의 경우는 p-value가 0.05보다 작기 때문에 두 포지션 간의 value 평균 차이는 존재한다고 볼 수 있다. <br>
또한 defender와 goalkeeper도 p-value가 0.05 보다 작기 때문에 두 포지션 간의 value 평균 차이는 존재한다고 볼 수 있다. <br>
defender와 midfielder / forward와 goalkeeper / goalkeeper와 midfielder의 p-value도 모두 0.05 보다 작기 때문에 두 포지션 간의 value 평균 차이는 존재한다고 볼 수 있다. <br>
하지만 forward와 midfielder의 경우 p-value가 0.05 보다 크기 때문에 두 포지션 간의 value 평균 차이는 존재하지 않는다고 볼 수 있다.

***
#### Q4) Preferred Foot(주로 사용하는 발)과 Position_Class(재범주화 된 포지션)변수에 
####      따라 Value(이적료)의 차이가 있는지를 알아보기 위해 이원배치분산분석을 
####      수행하고 결과를 해석하시오.

In [316]:
model2 = ols('Value ~ C(Position_Class) * Preferred_Foot', fifa).fit()

In [317]:
anova_lm(model2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Position_Class),3.0,4.081181e+09,1.360394e+09,41.912369,5.658225e-27
Preferred_Foot,1.0,1.522715e+08,1.522715e+08,4.691332,3.032930e-02
C(Position_Class):Preferred_Foot,3.0,4.736156e+08,1.578719e+08,4.863874,2.207249e-03
Residual,16634.0,5.399071e+11,3.245805e+07,NaN,NaN


먼저 position_class는 p-value가 0.05 보다 작기 때문에 귀무가설을 기각한다. 즉 '선수 포지션에 따라 value의 차이가 존재한다'고 볼 수 있다. <br>
preferred_foot의 경우도 p-value가 0.05 보다 작기 때문에 귀무가설을 기각하고 '주발에 따라 value의 차이가 존재한다'고 볼 수 있다.<br>
마지막으로 포지션과 주발의 교호작용에서도 p-value가 0.05 보다 작기 때문에 귀무가설을 기각하고 '주발과 포지션간의 상호작용의 효과가 있다'고 볼 수 있다.
***
#### Q5)Age, Overall, Wage, Height_cm, Weight_lb 가 Value에 영향을 미치는지 
####     알아보는 회귀분석을 단계적 선택법을 사용하여 수행하고 결과를 해석하시오. 

In [318]:
X = fifa[['Wage','Age','Overall','Height_cm','Weight_lb']]
y = fifa.Value

In [319]:
X['constant'] = 1

* 단계선택법

In [320]:
add_feature = ['constant']
aic_list = []
target_cols = list(X.columns[:-1])
i = 0
while True :
    i += 1
    print(f'{i}번째 탐색')
    if i == 1 :
        before_model = sm.OLS(y, X[['constant']]).fit()
        before_aic = model.aic
        for c in target_cols :
            after_model = sm.OLS(y, X[add_feature + [c]]).fit()
            after_aic = after_model.aic
            print(f'{c} 추가 되었을 때 AIC는 {after_aic}이다.')
            aic_list.append(after_aic)
        print(f'추가될 변수는 {target_cols[np.argmin(aic_list)]}')
        add_feature.append(target_cols[np.argmin(aic_list)])
        del target_cols[np.argmin(aic_list)]
    elif i != 1 :
        a = []
        for c in target_cols :
            before_model = sm.OLS(y, X[add_feature]).fit()
            before_aic = before_model.aic
            
            after_model = sm.OLS(y, X[add_feature + [c]]).fit()
            after_aic = after_model.aic
            
            print(f'{c} 추가 전은 {before_aic}이고 {c}추가 되었을 때 AIC는 {after_aic}')
            a.append(after_aic)
            
        if before_aic > np.min(a):
            print(f'추가될 feature는 {target_cols[np.argmin(a)]}')
            add_feature.append(target_cols[np.argmin(a)])
            del target_cols[np.argmin(a)]
        else :
            break

1번째 탐색
Wage 추가 되었을 때 AIC는 312636.7113986248이다.
Age 추가 되었을 때 AIC는 335101.9050948277이다.
Overall 추가 되었을 때 AIC는 326811.6430766084이다.
Height_cm 추가 되었을 때 AIC는 335199.3767194875이다.
Weight_lb 추가 되었을 때 AIC는 335166.9800023204이다.
추가될 변수는 Wage
2번째 탐색
Age 추가 전은 312636.7113986248이고 Age추가 되었을 때 AIC는 312462.32321630104
Overall 추가 전은 312636.7113986248이고 Overall추가 되었을 때 AIC는 310792.4128532369
Height_cm 추가 전은 312636.7113986248이고 Height_cm추가 되었을 때 AIC는 312623.2149041213
Weight_lb 추가 전은 312636.7113986248이고 Weight_lb추가 되었을 때 AIC는 312629.2657629237
추가될 feature는 Overall
3번째 탐색
Age 추가 전은 310792.4128532369이고 Age추가 되었을 때 AIC는 309173.1691867521
Height_cm 추가 전은 310792.4128532369이고 Height_cm추가 되었을 때 AIC는 310762.78845630656
Weight_lb 추가 전은 310792.4128532369이고 Weight_lb추가 되었을 때 AIC는 310700.69694111776
추가될 feature는 Age
4번째 탐색
Height_cm 추가 전은 309173.1691867521이고 Height_cm추가 되었을 때 AIC는 309167.6615284396
Weight_lb 추가 전은 309173.1691867521이고 Weight_lb추가 되었을 때 AIC는 309167.8762187488
추가될 feature는 Height_cm
5번째 탐색
Weight_lb 추

In [321]:
sm.OLS(y, X[add_feature]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                 1.572e+04
Date:                Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                        16:00:05   Log-Likelihood:            -1.5458e+05
No. Observations:               16642   AIC:                         3.092e+05
Df Residuals:                   16637   BIC:                         3.092e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant   -8690.8178    588.280    -14.773      0.000   -9843.908   -7537.727
Wage         184.1837      1.128    163.317      0.000     181.973     186.394
Overall      241.3450      4.002     60.310      0.000     233.501     249.189
Age         -202.1603      4.938    -40.936      0.000    -211.840    -192.480
Height_cm     -8.4446      3.082     -2.740      0.006     -14.486      -2.403
==============================================================================
Omnibus:                    17089.038   Durbin-Watson:                   1.407
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3525378.426
Skew:                           4.665   Prob(JB):                         0.00
Kurtosis:                      73.690   Cond. No.                     5.58e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

단계선택법 수행 결과 상수항, Wage, Age, Overall, Height_cm 변수들의 p-value가 0에 근사하여 통계적으로 유의한것으로 나타났다. 해당 모형의 결정계수는 0.791이고 수정된 결정계수 또한 0.791로 나타났다. 모형 자체에 대한 통계적 유의성은 F통계량을 살펴본 결과 p-value가 0.05 보다 작으므로 유의수준 0.05 하에서 통계적으로 모형이 유의한 것을 의미한다.<br>
최종 회귀식은 y = -8690.8178 + 184.1837 * Wage + 241.345 * Overall - 202.1603 * Age - 8.446 * Height_cm이다.